# NCEI 1991–2020 Climate Normals
Station: **USW00012921 — Houston Intercontinental Airport (IAH)**

This notebook loads the **daily** and **hourly** normals CSVs, processes them,
creates D3‑ready JSON exports, and includes simple sanity‑check charts using Matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

daily = pd.read_csv('USW00012921_daily.csv')
hourly = pd.read_csv('USW00012921_hourly.csv')
daily.head(), hourly.head()

## Normalize Columns
Convert daily to DOY and filter needed fields.

In [ ]:
# Daily normals: convert month/day to DOY
daily['DOY'] = pd.to_datetime({'year':2001,'month':daily['MO'],'day':daily['DY']}).dt.dayofyear

daily_temps = daily[['DOY','DLY-TMAX-NORMAL','DLY-TMIN-NORMAL','DLY-TAVG-NORMAL']].set_index('DOY')
daily_precip = daily[['DOY','DLY-PRCP-NORMAL']].set_index('DOY')
daily.head()

## Hourly Temperature Matrix 365×24

In [ ]:
hourly['TS'] = pd.to_datetime({'year':2001,'month':hourly['MO'],'day':hourly['DY']})
hourly['DOY'] = hourly['TS'].dt.dayofyear

hourly_temp = hourly.pivot(index='DOY', columns='HR', values='HLY-TEMP-NORMAL')
hourly_temp.head()

## Daily High / Low from Hourly

In [ ]:
high_from_hr = hourly.groupby('DOY')['HLY-TEMP-NORMAL'].max()
low_from_hr  = hourly.groupby('DOY')['HLY-TEMP-NORMAL'].min()
high_from_hr.head(), low_from_hr.head()

## Cloud Category Percentages per DOY

In [ ]:
# Cloud categories: pct Clear/Few/Scattered/Broken/Overcast
cloud_cols = [c for c in hourly.columns if c.startswith('HLY-CLOD-PCT')]
cloud_df = hourly[['DOY'] + cloud_cols].copy()

def cloud_category(row):
    vals = {cat:row[cat] for cat in cloud_cols}
    return max(vals, key=vals.get)

cloud_df['cat'] = cloud_df.apply(cloud_category, axis=1)
cloud_pct = (cloud_df.groupby(['DOY','cat']).size()
            .groupby(level=0).apply(lambda x: x/x.sum())
            .unstack(fill_value=0))
cloud_pct.head()

## Precipitation Chance per Day

In [ ]:
if 'HLY-PCPN-PCTH0XX' in hourly.columns:
    hourly['PRECIP_OCC'] = hourly['HLY-PCPN-PCTH0XX']
    precip_chance = hourly.groupby('DOY')['PRECIP_OCC'].mean()
else:
    precip_chance = daily_precip['DLY-PRCP-NORMAL'] > 0
precip_chance.head()

## Sliding 30‑Day Rainfall Sum

In [ ]:
rain = daily_precip['DLY-PRCP-NORMAL'].values
roll = np.concatenate([rain, rain, rain])
sliding = pd.Series(roll).rolling(30).sum().iloc[365:730].reset_index(drop=True)
sliding.head()

## Sanity Check Charts

In [ ]:
# Daily high/low
plt.figure(figsize=(12,4))
plt.plot(daily_temps['DLY-TMAX-NORMAL'], label='High')
plt.plot(daily_temps['DLY-TMIN-NORMAL'], label='Low')
plt.title('Daily High/Low Normals – USW00012921')
plt.legend(); plt.show()

In [ ]:
# Hourly temp example DOY=200
plt.figure(figsize=(10,4))
plt.plot(hourly_temp.loc[200])
plt.title('Hourly Temp – DOY 200'); plt.xlabel('Hour'); plt.ylabel('Temp');
plt.show()

## Export D3-Ready JSON

In [ ]:
out = {
    'hourly_temp': hourly_temp.round(1).to_dict(orient='list'),
    'daily_high': daily_temps['DLY-TMAX-NORMAL'].round(1).tolist(),
    'daily_low': daily_temps['DLY-TMIN-NORMAL'].round(1).tolist(),
    'daily_avg': daily_temps['DLY-TAVG-NORMAL'].round(1).tolist(),
    'precip_chance': precip_chance.tolist(),
    'cloud_pct': cloud_pct.to_dict(orient='index'),
    'sliding_rainfall': sliding.round(2).tolist(),
}

with open('USW00012921_normals.json','w') as f:
    json.dump(out,f,indent=2)
out.keys()